In [12]:
import numpy as np
import pandas as pd

options = pd.DataFrame({"Calls": [1175, 1200, 1225, 1250, 1275, 1300, 1325, 1350, 1375, 1400, 1425, 1450, 1500, 1550, 1575, 1600],
                       "Calls Prices": [225.40, 205.55, 186.20, 167.50, 149.15, 131.70, 115.25, 99.55, 84.90, 71.10, 58.70, 47.25, 29.25, 15.80, 11.10, 7.90],
                       "Puts":[1175, 1200, 1225, 1250, 1275, 1300, 1325, 1350, 1375, 1400, 1425, 1450, 1500, 1550, 1575, 1600],
                       "Puts Prices":[46.60, 51.55, 57.15, 63.30, 70.15, 77.70, 86.20, 95.30, 105.30, 116.55, 129.00, 143.20, 173.95, 210.80, 230.90, 252.40]})

options

,Calls,Calls Prices,Puts,Puts Prices
0,1175,225.40,1175,46.60
1,1200,205.55,1200,51.55
2,1225,186.20,1225,57.15
3,1250,167.50,1250,63.30
4,1275,149.15,1275,70.15
5,1300,131.70,1300,77.70
6,1325,115.25,1325,86.20
7,1350,99.55,1350,95.30
8,1375,84.90,1375,105.30
9,1400,71.10,1400,116.55


In [14]:
from scipy.stats import norm
from scipy.linalg import cholesky
def linear_solve_cholesky(A, b):
    U = cholesky(A)
    y = np.linalg.solve(np.transpose(U), b)
    x = np.linalg.solve(U, y)
    return x 
options['C-P'] = options['Calls Prices'] - options['Puts Prices']
ones = np.ones((16, 1))
strikes = -1 * np.matrix(options['Calls'].values).T
x_matrix = np.concatenate((ones, strikes), axis = 1)
y_vector = np.matrix(options['C-P'].values).T
#pvf, disc
res = linear_solve_cholesky(np.dot(np.transpose(x_matrix), x_matrix), np.dot(np.transpose(x_matrix), y_vector))
np.set_printoptions(suppress=True)
res

matrix([[1349.53656845],
        [   0.99642025]])

In [15]:
#calculate implied volatility
def BlackScholes(vol, K, T, PVF, DISC, call = True):
    d1 = np.log(PVF/(K*DISC))/(vol*np.sqrt(T)) + 0.5*(vol*np.sqrt(T))
    d2 = d1 - vol*np.sqrt(T)
    if (call == True):
        return PVF*norm.cdf(d1) - K*DISC*norm.cdf(d2)
    else: 
        return K*DISC*norm.cdf(-d2) - PVF*norm.cdf(-d1)
    
def implied_vol_call(Vm, k, t, pvf, disc, tol):    
    x0 = 0.25
    x_new = x0
    x_old = x0 - 1
    while (np.abs(x_new - x_old) > tol):
        x_old = x_new
        d1 = np.log(pvf/(k*disc))/(x_old*np.sqrt(t)) + 0.5*(x_old*np.sqrt(t))
        x_new = x_old - (BlackScholes(x_old, k, t, pvf, disc, call = True) - Vm)/((pvf*np.sqrt(t/(2*np.pi)))*np.exp(-0.5*(d1**2)))
    return x_new

def implied_vol_put(Vm, k, t, pvf, disc, tol):
    x0 = 0.25
    x_new = x0
    x_old = x0 - 1
    while (np.abs(x_new - x_old) > tol):
        x_old = x_new
        d1 = np.log(pvf/(k*disc))/(x_old*np.sqrt(t)) + 0.5*(x_old*np.sqrt(t))
        x_new = x_old - (BlackScholes(x_old, k, t, pvf, disc, call = False) - Vm)/((pvf*np.sqrt(t/(2*np.pi)))*np.exp(-0.5*(d1**2)))
    return x_new


T = 199/252
DISC = res[1]
PVF = res[0]

#cauculating...
options['Calls Imp'] = 0
options['Puts Imp'] = 0
for i in range(0, 16):
    options['Calls Imp'][i] = implied_vol_call(options['Calls Prices'][i], options['Calls'][i], T, PVF, DISC, 10**-6)
for i in range(0, 16):
    options['Puts Imp'][i] = implied_vol_put(options['Puts Prices'][i], options['Puts'][i], T, PVF, DISC, 10**-6)
options

<ipython-input-15-77a240c5d2e2>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  options['Calls Imp'][i] = implied_vol_call(options['Calls Prices'][i], options['Calls'][i], T, PVF, DISC, 10**-6)
<ipython-input-15-77a240c5d2e2>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  options['Puts Imp'][i] = implied_vol_put(options['Puts Prices'][i], options['Puts'][i], T, PVF, DISC, 10**-6)


,Calls,Calls Prices,Puts,Puts Prices,C-P,Calls Imp,Puts Imp
0,1175,225.40,1175,46.60,178.80,0.257327,0.257170
1,1200,205.55,1200,51.55,154.00,0.249649,0.249214
2,1225,186.20,1225,57.15,129.05,0.241945,0.241629
3,1250,167.50,1250,63.30,104.20,0.234443,0.233998
4,1275,149.15,1275,70.15,79.00,0.226301,0.226529
5,1300,131.70,1300,77.70,54.00,0.218641,0.219058
6,1325,115.25,1325,86.20,29.05,0.211476,0.211967
7,1350,99.55,1350,95.30,4.25,0.204086,0.204336
8,1375,84.90,1375,105.30,-20.40,0.196940,0.196645
9,1400,71.10,1400,116.55,-45.45,0.189444,0.189441
